In [ ]:
!pip install torch transformers accelerate git+https://github.acom/huggingface/transformers.git
!pip install git+https://github.com/deepseek-ai/janus.git

  Cloning https://github.acom/huggingface/transformers.git to /tmp/pip-req-build-wdbst7of
  Running command git clone --filter=blob:none --quiet https://github.acom/huggingface/transformers.git /tmp/pip-req-build-wdbst7of
  fatal: unable to access 'https://github.acom/huggingface/transformers.git/': Could not resolve host: github.acom
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.acom/huggingface/transformers.git /tmp/pip-req-build-wdbst7of did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.acom/huggingface/transformers.git /tmp/pip-req-build-wdbst7of did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
  Cloning https://github.com/deep

In [ ]:
import os
import torch
import numpy as np
import PIL.Image
from transformers import AutoModelForCausalLM
from janus.models import MultiModalityCausalLM, VLChatProcessor

torch.cuda.empty_cache()  # Free GPU memory

In [ ]:
# Model path
model_path = "deepseek-ai/Janus-Pro-1.3B"

# Load VLChatProcessor
vl_chat_processor = VLChatProcessor.from_pretrained(model_path, trust_remote_code=True)
tokenizer = vl_chat_processor.tokenizer

# Load model
vl_gpt = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,  # Efficient data type
    device_map="auto",           # Automatically assigns model to GPU
    trust_remote_code=True
).eval()

print("Model loaded successfully!")


OSError: deepseek-ai/Janus-Pro-1.3B is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
conversation = [
    {"role": "<|User|>", "content": "A stunning princess from Kabul in red, white traditional clothing, blue eyes, brown hair"},
    {"role": "<|Assistant|>", "content": ""},
]

sft_format = vl_chat_processor.apply_sft_template_for_multi_turn_prompts(
    conversations=conversation,
    sft_format=vl_chat_processor.sft_format,
    system_prompt="",
)
prompt = sft_format + vl_chat_processor.image_start_tag

In [ ]:
@torch.inference_mode()
def generate(
    mmgpt,
    vl_chat_processor,
    prompt,
    temperature=1,
    parallel_size=4,  # Reduced from 16 to 4
    cfg_weight=3.5,  # Slightly reduced
    image_token_num_per_image=256,  # Reduced from 576
    img_size=256,  # Reduced from 384
    patch_size=16,
):
    input_ids = vl_chat_processor.tokenizer.encode(prompt)
    input_ids = torch.LongTensor(input_ids).cuda()

    tokens = torch.zeros((parallel_size * 2, len(input_ids)), dtype=torch.int16).cuda()
    for i in range(parallel_size * 2):
        tokens[i, :] = input_ids
        if i % 2 != 0:
            tokens[i, 1:-1] = vl_chat_processor.pad_id

    inputs_embeds = mmgpt.language_model.get_input_embeddings()(tokens)

    generated_tokens = torch.zeros((parallel_size, image_token_num_per_image), dtype=torch.int16).cuda()

    for i in range(image_token_num_per_image):
        outputs = mmgpt.language_model.model(inputs_embeds=inputs_embeds, use_cache=True)
        hidden_states = outputs.last_hidden_state

        logits = mmgpt.gen_head(hidden_states[:, -1, :])
        logit_cond = logits[0::2, :]
        logit_uncond = logits[1::2, :]

        logits = logit_uncond + cfg_weight * (logit_cond - logit_uncond)
        probs = torch.softmax(logits / temperature, dim=-1)

        next_token = torch.multinomial(probs, num_samples=1)
        generated_tokens[:, i] = next_token.squeeze(dim=-1)

        next_token = torch.cat([next_token.unsqueeze(dim=1), next_token.unsqueeze(dim=1)], dim=1).view(-1)
        img_embeds = mmgpt.prepare_gen_img_embeds(next_token)
        inputs_embeds = img_embeds.unsqueeze(dim=1)

    dec = mmgpt.gen_vision_model.decode_code(
        generated_tokens.to(dtype=torch.int16),
        shape=[parallel_size, 8, img_size // patch_size, img_size // patch_size]
    )

    dec = dec.to(torch.float32).cpu().numpy().transpose(0, 2, 3, 1)
    dec = np.clip((dec + 1) / 2 * 255, 0, 255)

    os.makedirs('generated_samples', exist_ok=True)
    for i in range(parallel_size):
        save_path = os.path.join('generated_samples', f"img_{i}.jpg")
        PIL.Image.fromarray(dec[i].astype(np.uint8)).save(save_path)

    torch.cuda.empty_cache()

In [ ]:
generate(
    vl_gpt,
    vl_chat_processor,
    prompt,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 58.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 30.12 MiB is free. Process 35351 has 14.71 GiB memory in use. Of the allocated memory 14.06 GiB is allocated by PyTorch, and 540.75 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
import glob
from IPython.display import display

image_paths = glob.glob("generated_samples/*.jpg")
for img_path in image_paths:
    display(PIL.Image.open(img_path))